# libraries

In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from collections import deque

# random seed for reproducibility
np.random.seed(42)

# Loading the Karate Club Graph
G = nx.karate_club_graph()

# layout for consistent visualization
pos = nx.spring_layout(G, seed=42)

print(f"Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.")

Graph loaded: 34 nodes, 78 edges.


# Modularity Matrix 

In [ ]:
def get_modularity_matrix(G):
    A = nx.to_numpy_array(G)
    k = A.sum(axis=1) 
    m = G.number_of_edges()
    k_k_T = np.outer(k, k)
    B = A - (k_k_T / (2 * m))
    return B